# Prep incidence files

In [25]:
%matplotlib inline

In [26]:
import pandas as pd
import numpy as np

import utility.init as util

# Target master data

Date | Province_Id | Daily_Total | Daily_100k | N_Day_Rate | N_Day_Rate_Change | N_Day_Rate_Change_Sliding_Window



In [27]:
emr_cols = ['Date', 'Province_Id', 'Daily_Total', 'Daily_100k', 'N_Day_Rate','N_Day_Rate_Change','N_Day_Rate_Change_Sliding_Window']

# Helpers

In [28]:
# cumulative total to daily total
def daily_incident_rate(df, totalCol):
    df_c = df.copy()
    for province in df_c['Province_Id'].unique():
        mask = df_c['Province_Id'] == province 
        df_c.loc[mask, 'Daily_Total'] = df_c.loc[mask, totalCol].rolling(2).apply(lambda x : x[1]-x[0], raw=True )
    return df_c


def daily_per_100k(df):
    df_c = df.copy()
    for province in df_c['Province_Id'].unique():
        mask = df_c['Province_Id'] == province 
        df_c.loc[mask, 'Daily_100k'] = (df_c.loc[mask, 'Daily_Total'] / util.province_size[province]) * 100000
    return df_c

In [29]:
def calc_change_factor(window):
    return window[1]/window[0]

# Change rate
def incident_rate_change(df):
    df_c = df.copy()
    for province in df_c['Province_Id'].unique():
        mask = df_c['Province_Id'] == province 
        df_c.loc[mask, 'N_Day_Rate_Change'] = df_c.loc[mask, 'N_Day_Rate'].rolling(2).apply(calc_change_factor, raw=True )
    return df_c

# Add sliding window change rate
def sliding_window_change_rate(df):
    df_c = df.copy()
    for province in df_c['Province_Id'].unique():
        mask = df_c['Province_Id'] == province
        df_c.loc[mask, 'N_Day_Rate_Change_Sliding_Window'] = df_c.loc[mask, 'N_Day_Rate_Change'].rolling(util.change_rate_window_size).mean()
    return df_c

# Select belgian data

In [30]:
be_provs = ['Liège','Limburg']
be_df = pd.DataFrame(columns=emr_cols)

be_raw = pd.read_csv(util.data_dir + 'orig/COVID19BE_CASES_AGESEX.csv', sep=',')
be_raw = be_raw[be_raw['PROVINCE'].isin(be_provs)]
be_raw = be_raw[be_raw.DATE.notnull()]
be_raw.loc[be_raw['PROVINCE']=='Liège','Province_Id'] = 11
be_raw.loc[be_raw['PROVINCE']=='Limburg','Province_Id'] = 12

be_raw = be_raw.rename(columns={"DATE":"Date", "CASES": "Daily_Total"})

be_raw = be_raw.groupby(['Date','Province_Id']).sum().reset_index()
be_df = be_df.append(be_raw, ignore_index=True)

be_total = be_raw.groupby(['Date']).sum().reset_index()
be_total['Province_Id'] = 10
be_df = be_df.append(be_total, ignore_index=True)

# Select dutch data

In [31]:
nl_provs = ['Limburg']
nl_df = pd.DataFrame(columns=emr_cols)

nl_raw = pd.read_csv(util.data_dir + 'orig/COVID-19_aantallen_gemeente_cumulatief.csv', sep=';')
nl_raw = nl_raw[nl_raw['Province'].isin(nl_provs)]
nl_raw = nl_raw.rename(columns={"Date_of_report":"Date","Total_reported":"Daily_Total"})
nl_raw = nl_raw.groupby(['Date']).sum().reset_index()
nl_raw['Province_Id'] = 20

nl_raw = daily_incident_rate(nl_raw, 'Daily_Total')
nl_raw['Date'] = nl_raw['Date'].str[:10]
nl_raw['Daily_Total'] = nl_raw['Daily_Total'].shift(periods=-1)
nl_raw = nl_raw[nl_raw.Daily_Total.notnull()]

nl_df = nl_df.append(nl_raw, ignore_index=True)

# Select DE data

In [32]:
de_df = pd.DataFrame(columns=emr_cols)
de_raw = pd.read_csv(util.data_dir + 'orig/RKI_COVID19.csv', sep=',')

de_raw = de_raw[de_raw['Landkreis'].isin(util.province_id.keys())]
for lk, id in util.province_id.items():
    de_raw.loc[de_raw['Landkreis'] == lk, 'Province_Id'] = id

de_raw = de_raw.rename(columns={"Refdatum":"Date","AnzahlFall":"Daily_Total"})
de_raw['Date'] = de_raw['Date'].str[:4] + '-' + de_raw['Date'].str[5:7]+ '-' + de_raw['Date'].str[8:10]

de_raw = de_raw.groupby(['Date', 'Province_Id']).sum().reset_index()

de_df = de_df.append(de_raw, ignore_index=True)

de_total = de_raw.groupby(['Date']).sum().reset_index()
de_total['Province_Id'] = 30
de_df = de_df.append(de_total, ignore_index=True)


In [33]:
emr_df = pd.DataFrame(columns=emr_cols)

emr_df = emr_df.append(be_df, ignore_index=True)
emr_df = emr_df.append(nl_df, ignore_index=True)
emr_df = emr_df.append(de_df, ignore_index=True)

emr_df = emr_df.loc[emr_df['Date'] >= util.timeframe_start]
emr_df = emr_df.loc[emr_df['Date'] <= util.timeframe_end]
emr_df = emr_df.loc[:,:'N_Day_Rate_Change_Sliding_Window']
emr_df = emr_df.sort_values(by=['Date', 'Province_Id'])

# Add derived attributes

In [34]:
emr_df = emr_df.sort_values(by=['Date'])

# scale to 100k
emr_df = daily_per_100k(emr_df)

# N-Day Rate
emr_df['N_Day_Rate'] = emr_df['Daily_100k'].rolling(util.incident_window_size).sum()

# Rate Change
emr_df = incident_rate_change(emr_df)

emr_df = sliding_window_change_rate(emr_df)

emr_df.tail(30)

,Date,Province_Id,Daily_Total,Daily_100k,N_Day_Rate,N_Day_Rate_Change,N_Day_Rate_Change_Sliding_Window
4895,2021-11-12,33.0,55,21.446007,176.222742,0.325071,1.135855
1238,2021-11-12,12.0,1094,124.690837,284.243910,0.896152,1.069785
1237,2021-11-12,11.0,1263,113.804289,368.005475,1.198333,1.217409
1885,2021-11-12,10.0,2357,118.610889,458.318214,1.494225,1.297398
2508,2021-11-12,20,1192.0,106.820086,541.754428,1.79453,1.193859
4898,2021-11-13,32.0,33,12.446255,524.378412,1.763592,1.160185
4897,2021-11-13,31.0,175,31.43914,529.257503,1.781172,1.131333
5588,2021-11-13,30.0,279,21.923828,529.735324,2.625894,1.236837
4899,2021-11-13,34.0,71,36.530338,441.574825,1.448491,1.145617
1240,2021-11-13,12.0,643,73.287211,401.057746,1.410963,1.169181


# Save to disc

In [35]:
emr_df.to_csv(util.emr_infection_data)